## Import Library And Setup Folder

In [ ]:
import os
import random
import gdown
import tensorflow as tf
from tensorflow import keras
from matplotlib import gridspec
from matplotlib.patches import Rectangle
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers
# from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix
import seaborn as sn
import pandas as pd
import pathlib
import subprocess
from io import BytesIO
from PIL import Image
from IPython.display import display
import ipywidgets as widgets
import math

INPUT_SIZE = (224, 224)

WORKING_PATHS = {
    'BASE': os.path.join('development'),
    'DATASET': os.path.join('development', 'dataset'),
    'TRAIN_DATASET': os.path.join('development', 'dataset', 'train'),
    'TEST_DATASET': os.path.join('development', 'dataset', 'test'),
    'VALID_DATASET': os.path.join('development', 'dataset', 'val'),
    'OUTPUT': os.path.join('development', 'output'), 
    'EXPORT': os.path.join('development', 'output', 'my-model', 'export', 'BISINDO'), 
    'TFLITE':os.path.join('development', 'output', 'my-model', 'tflite'), 
 }

In [2]:
# Create working dirs
for path in WORKING_PATHS.values():
    if not os.path.exists(path):
        if os.name == 'posix':
            !mkdir -p {path}
        if os.name == 'nt':
            !mkdir {path}

## Data Argumentation

In [ ]:
import os
import random
import cv2
import numpy as np
from PIL import Image, ImageEnhance
import imutils

# Target jumlah data per kelas
TARGET_SIZE = 10000
FINAL_SIZE = (224, 224)
IMG_SIZE = (224, 224)
AUGMENTATION_FOLDERS = ["resize", "brightness", "translate", "zoom", "rotate"]

# Fungsi untuk augmentasi satu gambar
def augment_image(image, folder, index):
    augmented_images = []

    # Resize
    resized_image = cv2.resize(image, FINAL_SIZE)
    augmented_images.append(resized_image)

    # Brightness
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    enhancer = ImageEnhance.Brightness(pil_image)
    for factor in [0.75, 1.0, 1.25, 1.5]:
        bright_img = enhancer.enhance(factor)
        augmented_images.append(cv2.cvtColor(np.array(bright_img), cv2.COLOR_RGB2BGR))

    # Translate
    for x_shift in range(-10, 11, 5):
        for y_shift in range(-10, 11, 5):
            M = np.float32([[1, 0, x_shift], [0, 1, y_shift]])
            translated = cv2.warpAffine(image, M, IMG_SIZE)
            translated = cv2.resize(translated, FINAL_SIZE)
            augmented_images.append(translated)

    # Zoom
    for scale in [12, 24, 36]:
        zoom_crop = image[scale:-scale, scale:-scale]
        zoom_resized = cv2.resize(zoom_crop, FINAL_SIZE)
        augmented_images.append(zoom_resized)

    # Rotate
    for angle in range(-45, 50, 10):
        rotated = imutils.rotate_bound(image, angle)
        rotated = cv2.resize(rotated, FINAL_SIZE)
        augmented_images.append(rotated)

    return augmented_images

# Proses setiap folder
for folder in sorted(os.listdir(datasets_dir)):
    folder_path = os.path.join(datasets_dir, folder)
    print(f"Processing folder: {folder} (Original images: {len(os.listdir(folder_path))})")

    # Buat folder target
    output_dir = os.path.join("./kurang", folder)
    os.makedirs(output_dir, exist_ok=True)

    # Ambil semua gambar asli
    original_images = sorted(os.listdir(folder_path))
    num_original = len(original_images)

    # Jika gambar asli >= TARGET_SIZE, pilih secara acak hingga jumlahnya pas
    if num_original >= TARGET_SIZE:
        selected_images = random.sample(original_images, TARGET_SIZE)
        for i, image_name in enumerate(selected_images):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, FINAL_SIZE)
            output_path = os.path.join(output_dir, f"{folder}_img_{i:05d}.jpg")
            cv2.imwrite(output_path, image)
    else:
        # Augmentasi diperlukan
        total_augmented = 0
        augmentations_needed = TARGET_SIZE - num_original

        for i, image_name in enumerate(original_images):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, IMG_SIZE)

            # Augment gambar ini
            augmented_images = augment_image(image, folder, i)
            for aug_image in augmented_images:
                if total_augmented >= augmentations_needed:
                    break
                output_path = os.path.join(output_dir, f"{folder}_img_{total_augmented:05d}.jpg")
                cv2.imwrite(output_path, aug_image)
                total_augmented += 1

            if total_augmented >= augmentations_needed:
                break

        # Tambahkan gambar asli ke folder target
        for i, image_name in enumerate(original_images):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            image = cv2.resize(image, FINAL_SIZE)
            output_path = os.path.join(output_dir, f"{folder}_img_{total_augmented + i:05d}.jpg")
            cv2.imwrite(output_path, image)

    print(f"Total images for folder {folder}: {TARGET_SIZE}")

print("Augmentation completed!")


In [ ]:
print("Training labels:", os.listdir(WORKING_PATHS['TRAIN_DATASET']))
print("Validation labels:", os.listdir(WORKING_PATHS['VALID_DATASET']))

## Visualisasi Jumlah data dan Sampel Gamabar

In [ ]:
list_label = os.listdir(WORKING_PATHS['TRAIN_DATASET'])

train_data_count = {label: 0 for label in list_label}
test_data_count = {label: 0 for label in list_label}
TRAIN_IMG_MIN = float('inf')

for label in list_label:
    train_label_path = os.path.join(WORKING_PATHS['TRAIN_DATASET'], label)
    test_label_path = os.path.join(WORKING_PATHS['VALID_DATASET'], label)

    train_data_count[label] = len(os.listdir(train_label_path))
    test_data_count[label] = len(os.listdir(test_label_path))

    TRAIN_IMG_MIN = min(TRAIN_IMG_MIN, train_data_count[label])

y_positions = np.arange(len(list_label))
train_counts = list(train_data_count.values())
test_counts = list(test_data_count.values())

fig, axes = plt.subplots(1, 2, figsize=(18, 6))
fig.suptitle('Training and Validation Data Distribution', fontsize=16)

axes[0].bar(y_positions, train_counts, alpha=0.7, color='blue', width=0.6)
axes[0].set_xticks(y_positions)
axes[0].set_xticklabels(list_label, fontsize=12)
axes[0].set_title('Training Data', fontsize=14)
axes[0].set_ylabel('Number of Images', fontsize=12)
axes[0].set_xlabel('Labels', fontsize=12)

axes[1].bar(y_positions, test_counts, alpha=0.7, color='orange', width=0.6)
axes[1].set_xticks(y_positions)
axes[1].set_xticklabels(list_label, fontsize=12)
axes[1].set_title('Validation Data', fontsize=14)
axes[1].set_ylabel('Number of Images', fontsize=12)
axes[1].set_xlabel('Labels', fontsize=12)

plt.subplots_adjust(wspace=0.4)
plt.show()

In [ ]:
seed = random.randint(0, TRAIN_IMG_MIN)

columns = 3
rows = math.ceil(len(list_label) / columns)

figure_size = 4
fig = plt.figure(figsize=(figure_size * columns, figure_size * rows))
fig.suptitle('Randomly Selected Training Preview', fontsize=20, fontweight='bold', y=0.98)

grid = gridspec.GridSpec(rows, columns, wspace=0.5, hspace=0.8)

for idx, label in enumerate(list_label):
    label_dir = os.path.join(WORKING_PATHS['TRAIN_DATASET'], label)
    random_image = random.choice(os.listdir(label_dir))
    image_path = os.path.join(label_dir, random_image)
    
    img = image.load_img(image_path)
    img_resized = img.resize(INPUT_SIZE)

    ax = fig.add_subplot(grid[idx])
    ax.imshow(img_resized)
    ax.set_title(label, fontsize=12, fontweight='medium', color='navy')
    ax.axis('off')

    frame_color = 'black'
    border_thickness = 2
    ax.add_patch(Rectangle(
        (0, 0), INPUT_SIZE[0], INPUT_SIZE[1],
        linewidth=border_thickness,
        edgecolor=frame_color,
        facecolor='none'
    ))

plt.subplots_adjust(top=0.92, left=0.05, right=0.95, bottom=0.05)
plt.show()

## Data Preprocessing

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale = 1 / 255,
    fill_mode = 'nearest'
)
validation_datagen = ImageDataGenerator(rescale = 1 / 255)

train_generator = train_datagen.flow_from_directory(
    WORKING_PATHS['TRAIN_DATASET'],
    target_size = INPUT_SIZE,
    class_mode = 'categorical'
)
validation_generator = validation_datagen.flow_from_directory(
    WORKING_PATHS['VALID_DATASET'],
    target_size = INPUT_SIZE,
    class_mode = 'categorical'
)

In [ ]:
LABELS = list(train_generator.class_indices.keys())
NUM_CLASSES = len(LABELS)
print(NUM_CLASSES)

## Load Pre Train Model dan Fine Tuning

In [9]:
pre_trained_model = MobileNetV2(
    weights = 'imagenet', 
    input_shape = (INPUT_SIZE[0], INPUT_SIZE[1], 3), 
    include_top = False, 
)

pre_trained_model.trainable = False

In [10]:
#Fine Tuning
pre_trained_model.trainable = True

for layer in pre_trained_model.layers[:100]:
    layer.trainable = False

## Bulid Model

In [ ]:
# x = layers.Flatten()(pre_trained_model.output)
x = layers.Dropout(0.3)(pre_trained_model.output)
x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = layers.Dropout(0.6)(x)
x = layers.Dense(NUM_CLASSES, activation='softmax')(x) 

model = Model(inputs = pre_trained_model.input, outputs = x)


model.summary()

## Membuat Callback

In [12]:
class CustomCallback(Callback):
    def __init__(self, target_accuracy=0.9, max_val_loss=0.2):
        super().__init__()
        self.target_accuracy = target_accuracy
        self.max_val_loss = max_val_loss

    def on_epoch_end(self, epoch, logs=None):
        # Memeriksa apakah accuracy >= 0.95 dan validation loss <= 0.1
        val_loss = logs.get('val_loss')
        accuracy = logs.get('accuracy')

        if accuracy >= self.target_accuracy and val_loss <= self.max_val_loss:
            print(f"\nEpoch {epoch+1}: Accuracy is {accuracy:.4f} and validation loss is {val_loss:.4f}. Stopping training.")
            self.model.stop_training = True  # Menghentikan pelatihan

In [13]:
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=3, 
    restore_best_weights=True
)

custom_callback = CustomCallback(target_accuracy=0.95, max_val_loss=0.1)

## Train Model

In [ ]:
model.compile(
    optimizer = Adam(learning_rate=0.0001),
    loss = 'categorical_crossentropy', 
    metrics = ['accuracy']
)

history = model.fit(
    train_generator, 
    validation_data = validation_generator, 
    epochs = 10, 
    # validation_steps = 5,
    callbacks=[early_stopping, custom_callback],
    verbose = 1,
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

fig, axs = plt.subplots(2, 1, figsize = (5, 6))
fig.suptitle('Training and Validation plotting')
axs[0].plot(epochs, acc, 'r', label = 'Training accuracy')
axs[0].plot(epochs, val_acc, 'b', label = 'Validation accuracy')
axs[0].set_title('Training')
axs[0].legend()
axs[1].plot(epochs, loss, 'r', label = 'Training Loss')
axs[1].plot(epochs, val_loss, 'b', label = 'Validation Loss')
axs[1].set_title('Loss')
axs[1].legend()
plt.show()

## Evaluasi Model

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1.0 / 255)

test_generator = test_datagen.flow_from_directory(
    WORKING_PATHS['TEST_DATASET'],
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


In [ ]:
results = model.evaluate(test_generator, verbose=1)  # Atau test_dataset
print(f"Loss: {results[0]:.4f}, Accuracy: {results[1]:.4f}")


In [27]:
def selectRandomImage(labels = None):
    if labels == None:
        seed = random.randint(1, NUM_CLASSES)
        label_seed = LABELS[seed - 1]
    else:
        seed = random.randint(1, len(labels))
        label_seed = labels[seed - 1]
    
    path = os.path.join(WORKING_PATHS['TEST_DATASET'], label_seed)
    test_dir = os.listdir(path)
    test_dir_num = len(test_dir)
    file_name = os.listdir(path)[random.randint(0, test_dir_num - 1)]
    return (os.path.join(path, file_name), file_name, label_seed)

In [28]:
def create_result_plot(prediction_list, prediction_label, actual_label, file_name, img):
    # Create figure and axes
    fig = plt.figure(figsize=(12, 6))
    grid = fig.add_gridspec(1, 2, wspace=0.4)
    fig.suptitle(f'Image "{file_name}" Predicted as: {prediction_label}', fontsize=16, fontweight='bold', y=1.02)

    # Set background color based on prediction correctness (softer colors)
    bg_color = '#C6E2FF' if prediction_label == actual_label else '#F5B7B1'  # Light blue for correct, soft red for incorrect
    fig.patch.set_facecolor(bg_color)

    # Plot actual image with title
    ax1 = fig.add_subplot(grid[0, 0])
    ax1.imshow(img)
    ax1.axis('off')  # Hide axes for a cleaner appearance
    ax1.set_title(f'Actual: {actual_label}', fontsize=14, color='midnightblue', fontweight='medium')

    # Plot prediction probabilities
    ax2 = fig.add_subplot(grid[0, 1])
    ax2.bar(range(len(prediction_list)), prediction_list, color='#A9D0F5', edgecolor='#AED6F1')  # Soft pastel blue
    ax2.set_title('Class Probabilities', fontsize=14, fontweight='medium')
    ax2.set_ylabel('Probability', fontsize=12)
    ax2.set_xlabel('Classes', fontsize=12)
    ax2.set_xticks(range(len(prediction_list)))
    ax2.set_xticklabels(LABELS, rotation=45, fontsize=10)  # Rotate for better visibility

    # Enhance layout
    plt.subplots_adjust(top=0.85, bottom=0.2, left=0.1, right=0.95)
    plt.show()


In [ ]:
# file_path, file_name, label = selectRandomImage()
file_path, file_name, label = selectRandomImage(['N'])
img = image.load_img(file_path, target_size = INPUT_SIZE)
x = image.img_to_array(img)
x = np.expand_dims(x, axis = 0)
x = preprocess_input(x)

prediction = model.predict(x, batch_size = 10)
index = int(prediction[0].argmax(axis = -1))

create_result_plot(prediction[0].reshape(NUM_CLASSES), LABELS[index], label, file_name, img)

In [ ]:
MAX_FILES_PER_LABEL = 100

y_true = []
y_pred = []

for label in LABELS:
    path = os.path.join(WORKING_PATHS['TEST_DATASET'], label)
    files = os.listdir(path)[:MAX_FILES_PER_LABEL]
    
    for file_name in files:
        file_loc = os.path.join(path, file_name)
        
        img = image.load_img(file_loc, target_size=INPUT_SIZE)
        x = preprocess_input(np.expand_dims(image.img_to_array(img), axis=0))

        
        index = model.predict(x, batch_size=1).argmax(axis=-1)[0]
        y_true.append(label)
        y_pred.append(LABELS[index])

mat = confusion_matrix(y_true, y_pred, labels=LABELS)
mat_norm = mat / mat.sum(axis=1, keepdims=True)

In [ ]:
df_cm = pd.DataFrame(mat_norm, index=LABELS, columns=LABELS)

plt.figure(figsize=(20, 15))
ax = sn.heatmap(
    df_cm,
    annot=True,
    fmt=".2f",
    cmap="coolwarm",
    annot_kws={"size": 10},
    linewidths=0.5,
    linecolor="black"
)


ax.set_title("Test Confusion Matrix", fontsize=20, pad=20)
ax.set_xlabel("Predicted Label", fontsize=16, labelpad=20)
ax.set_ylabel("True Label", fontsize=16, labelpad=20)


plt.xticks(rotation=45, fontsize=12, ha="right")
plt.yticks(rotation=0, fontsize=12)


plt.tight_layout()
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(y_true, y_pred, target_names=LABELS)

print(report)


## Save Model dan Convert TFLITE

In [ ]:
# Define the export directory
export_dir = os.path.join(os.getcwd(), WORKING_PATHS['EXPORT'], 'BISINDO')  # Change 'bisindo' to your desired model name

# Save the model in the SavedModel format
tf.saved_model.save(model, export_dir)

In [20]:
converter = tf.lite.TFLiteConverter.from_saved_model(WORKING_PATHS['EXPORT'])
tflite_model = converter.convert()

# Save the model.
with open(os.path.join(WORKING_PATHS['TFLITE'], 'mobilenetv2-bisindo-model.tflite'), 'wb') as f:
  f.write(tflite_model)

In [21]:
converter = tf.lite.TFLiteConverter.from_saved_model(WORKING_PATHS['EXPORT'])
tflite_model = converter.convert()

In [ ]:
tflite_model_file = pathlib.Path('mobilenetv2-bisindo-model.tflite.tflite')
tflite_model_file.write_bytes(tflite_model)
print(f"Model disimpan di: {tflite_model_file}")

In [ ]:
import tensorflow as tf

model = tf.keras.models.load_model(r"C:\bahasa-isyarat\Bangkit-Capstone\development\output\my-model\export\BISINDO\mobilenetv2-bisindo-model.tflite.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("coba.tflite", "wb").write(tflite_model)

In [ ]:
import tensorflow as tf

# Muat model dari file .h5
h5_model_path = r"C:\bahasa-isyarat\Bangkit-Capstone\development\output\my-model\export\BISINDO\mobilenetv2-bisindo-model.tflite.h5"
model = tf.keras.models.load_model(h5_model_path)

# Konversi model ke format TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Simpan model .tflite
tflite_model_path = r"C:\bahasa-isyarat\Bangkit-Capstone\model\coba.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print("Model berhasil dikonversi ke format TFLite.")


In [ ]:
# Define the export directory
export_dir = os.path.join(os.getcwd(), WORKING_PATHS['EXPORT'], 'mobilenetv2-bisindo-model.h5')  # Ubah ke nama file yang diinginkan

# Save the model in H5 format
model.save(export_dir, save_format='h5')


## Test Model

In [ ]:
import ipywidgets as widgets
import tensorflow as tf
import numpy as np
from io import BytesIO
from IPython.display import display

# Inisialisasi penghitung prediksi benar dan salah
correct_predictions = 0
incorrect_predictions = 0

# Memuat model TFLite
try:
    interpreter = tf.lite.Interpreter(model_path="mobilenetv2-bisindo-model.tflite")
    interpreter.allocate_tensors()
    print("Model berhasil dimuat.")
except Exception as e:
    print(f"Error saat memuat model: {e}")

# Mendapatkan detail input dan output model
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Periksa ukuran input model
input_shape = input_details[0]['shape']
print(f"Input model shape: {input_shape}")

# Daftar kelas (A-Z)
classes = [chr(i) for i in range(ord('A'), ord('Z') + 1)]
print(f"Kelas model: {classes}")

# Membuat widget untuk mengunggah file
uploader = widgets.FileUpload(accept="image/*", multiple=True)
clear_button = widgets.Button(description="Clear Output", button_style='danger')  # Tombol Clear Output
out = widgets.Output()

# Menampilkan widget
box = widgets.VBox([uploader, clear_button, out])
display(box)

def clear_output_area(_):
    """Fungsi untuk membersihkan output prediksi."""
    global correct_predictions, incorrect_predictions
    with out:
        out.clear_output()
        correct_predictions = 0
        incorrect_predictions = 0
        print("Output telah dibersihkan.")
        print(f"Jumlah prediksi benar: {correct_predictions}")
        print(f"Jumlah prediksi salah: {incorrect_predictions}")

clear_button.on_click(clear_output_area)

def preprocess_image(file, target_shape):
    """Fungsi untuk memuat dan memproses gambar agar sesuai dengan input model."""
    try:
        # Memuat gambar
        image = tf.image.decode_image(file.read(), channels=3)
        # Ubah ukuran gambar sesuai target
        image = tf.image.resize(image, (target_shape[1], target_shape[2]))
        # Normalisasi gambar ke rentang [0, 1]
        image = image / 255.0
        # Tambahkan dimensi batch
        image = np.expand_dims(image, axis=0).astype(np.float32)
        return image
    except Exception as e:
        print(f"Error saat memproses gambar: {e}")
        return None

def file_predict(filename, file, out):
    global correct_predictions, incorrect_predictions  # Akses variabel global

    try:
        # Preproses gambar
        image = preprocess_image(file, input_shape)
        if image is None:
            raise ValueError("Gagal memproses gambar.")

        # Melakukan inferensi menggunakan model TFLite
        interpreter.set_tensor(input_details[0]['index'], image)
        interpreter.invoke()
        prediction = interpreter.get_tensor(output_details[0]['index'])[0]

        with out:
            # Menampilkan hasil prediksi
            print(f"\nFile: {filename}")
            print(f"Model output: {prediction}")

            # Ambil kelas dengan probabilitas tertinggi
            prediction_index = np.argmax(prediction)
            predicted_class = classes[prediction_index]
            print(f"Prediksi: {filename} adalah huruf {predicted_class}")

            # Ekstrak label huruf dari nama file (misalnya 'Huruf-B-36.jpg' -> 'B')
            true_class = filename.split('-')[1]  # Ambil huruf dari nama file (misalnya 'Huruf-B-36' -> 'B')
            if predicted_class == true_class:
                correct_predictions += 1
                print(f"Prediksi benar!")
            else:
                incorrect_predictions += 1
                print(f"Prediksi salah! Seharusnya: {true_class}")

            # Tampilkan jumlah prediksi yang benar dan salah
            print(f"\nJumlah prediksi benar: {correct_predictions}")
            print(f"Jumlah prediksi salah: {incorrect_predictions}")

    except Exception as e:
        with out:
            print(f"Error saat memproses file {filename}: {e}")

def on_upload_change(change):
    """Fungsi untuk menangani file yang diunggah dan menjalankan prediksi."""
    items = change.new
    for item in items:
        try:
            print(f"Mengunggah file: {item.name}")
            file_jpgdata = BytesIO(item.content)
            file_predict(item.name, file_jpgdata, out)
        except Exception as e:
            print(f"Error saat menangani file {item.name}: {e}")

# Menghubungkan widget uploader ke fungsi
uploader.observe(on_upload_change, names='value')
print("Widget uploader terhubung. Silakan unggah file gambar.")
